# Preconfiguration

In [1]:
! rm -rf LoanStats_web*

In [2]:
! gsutil cp gs://grizzy-lab/LoanStats_web.csv ./

Copying gs://grizzy-lab/LoanStats_web.csv...
\ [1 files][  1.1 GiB/  1.1 GiB]   42.8 MiB/s                                   
Operation completed over 1 objects/1.1 GiB.                                      


In [3]:
! hdfs dfs -rm -r /rawzone

rm: `/rawzone': No such file or directory


In [4]:
! hdfs dfs -mkdir -p /rawzone/

In [5]:
! hdfs dfs -put LoanStats_web.csv /rawzone/

In [6]:
import time as t

##### Pre-Configuration of Spark

function sparknotebook()
{
export SPARK_LOCAL_IP="192.168.10.1"
export PYSPARK_DRIVER_PYTHON=jupyter
export PYSPARK_DRIVER_PYTHON_OPTS="notebook --no-browser --NotebookApp.token='' --allow-root --NotebookApp.ip='*' --NotebookApp.port=8880"
nohup $SPARK_HOME/bin/pyspark --master spark://192.168.10.1:7077
}

*** NO ANY CRONTAB JOB ***

In [7]:
! rm -rf metastore_db/

In [8]:
! pip install pandas

In [9]:
from pyspark.sql import functions as F

In [10]:
spark

In [11]:
raw_LendingClubWeb_df = spark.read.format('csv').\
option('header','true').option('mode','DROPMALFORMED')\
.load('/rawzone/LoanStats_web.csv')

##### Get row count

In [12]:
raw_LendingClubWeb_df.count()

1432466

##### Get column count

In [13]:
len(raw_LendingClubWeb_df.columns)

144

##### Get Data Type

In [14]:
raw_LendingClubWeb_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: string (nullable = true)
 |-- funded_amnt: string (nullable = true)
 |-- funded_amnt_inv: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string 

##### Get Freq. with Various Data Group for Detection of Some Outliers and other Noises.

In [15]:
raw_LendingClubWeb_df.select(['purpose']).groupBy('purpose').count().show()

+--------------------+------+
|             purpose| count|
+--------------------+------+
|             wedding|     7|
|                null|    26|
|               other| 93997|
|      small_business| 15032|
|  debt_consolidation|791531|
|         credit_card|335091|
|              moving|  9733|
|            vacation| 10579|
| and also pay off...|     1|
|    renewable_energy|   910|
|               house|  9893|
|                 car| 15288|
|      major_purchase| 32649|
|             medical| 18807|
|    home_improvement| 98922|
+--------------------+------+



In [16]:
raw_LendingClubWeb_df.groupBy('purpose')\
.agg(F.avg(F.col("loan_amnt")),F.sum(F.col("loan_amnt"))).show()

+--------------------+------------------+---------------+
|             purpose|    avg(loan_amnt)| sum(loan_amnt)|
+--------------------+------------------+---------------+
|             wedding|11967.857142857143|        83775.0|
|                null|              null|           null|
|               other|10855.311339723608|    1.0203667E9|
|      small_business|17023.697778073445|   2.55900225E8|
|  debt_consolidation|16461.308685319967|1.3029636125E10|
|         credit_card|15365.308005884968|  5.148776425E9|
|              moving| 8649.018802013768|      8.41809E7|
|            vacation| 6489.663484261272|     6.865415E7|
| and also pay off...|            8800.0|         8800.0|
|    renewable_energy|11595.631868131868|    1.0552025E7|
|               house| 16221.37875265339|     1.604781E8|
|                 car| 9686.968537414967|   1.48094375E8|
|      major_purchase|13169.705810285153|   4.29977725E8|
|             medical|  9730.98314457383|     1.830106E8|
|    home_impr

In [17]:
raw_LendingClubWeb_df.select(['home_ownership']).groupBy('home_ownership').count().show()

+--------------+------+
|home_ownership| count|
+--------------+------+
|          null|    26|
|           OWN|170888|
|          RENT|556961|
|      MORTGAGE|701646|
|           ANY|  2940|
|          NONE|     5|
+--------------+------+



In [18]:
raw_LendingClubWeb_df.select(['grade']).groupBy('grade').count().show()

+-----+------+
|grade| count|
+-----+------+
|    F| 19480|
| null|    26|
|    E| 68353|
|    B|424110|
|    D|191137|
|    C|421101|
|    A|302988|
|    G|  5271|
+-----+------+



In [19]:
raw_LendingClubWeb_df.select(['emp_title']).groupBy('emp_title').count().show()

+--------------------+-----+
|           emp_title|count|
+--------------------+-----+
|                 CSR|  694|
| Director of nursing|   43|
|           Physician| 1337|
|Digital Marketing...|   69|
|           machinist|  429|
|      Content Editor|    7|
|    Dedicated Driver|    4|
|         agent staff|    1|
|    laborer-operator|    1|
|           Scientist|  317|
|Journeyman Electr...|  146|
|Player Services C...|    1|
|PATIENT COORDINATOR |    1|
|          lieutenant|   26|
|Training Specialist |   34|
| Field Sales Manager|   24|
|Commercial Truck ...|   35|
|                 SGT|   31|
|    Reset Specialist|    6|
|Client Service Co...|   10|
+--------------------+-----+
only showing top 20 rows



In [20]:
raw_LendingClubWeb_df.select(['emp_length']).groupBy('emp_length').count().show()

+----------+------+
|emp_length| count|
+----------+------+
|   9 years| 46393|
|   5 years| 88686|
|      null|    26|
|    1 year| 95926|
|       n/a|108470|
|   2 years|130251|
|   7 years| 48965|
|   8 years| 51222|
|   4 years| 86767|
|   6 years| 62261|
|   3 years|116802|
| 10+ years|478304|
|  < 1 year|118393|
+----------+------+



In [21]:
raw_LendingClubWeb_df.select(['loan_status']).groupBy('loan_status').count().show()

+------------------+------+
|       loan_status| count|
+------------------+------+
|        Fully Paid|583755|
|           Default|  1419|
|              null|    26|
|   In Grace Period|  6136|
|       Charged Off|152999|
|Late (31-120 days)| 19455|
|           Current|663690|
| Late (16-30 days)|  4986|
+------------------+------+



In [22]:
raw_LendingClubWeb_df.select(['term']).groupBy('term').count().show()

+----------+-------+
|      term|  count|
+----------+-------+
| 36 months|1036235|
|      null|     26|
| 60 months| 396205|
+----------+-------+



In [23]:
raw_LendingClubWeb_df.select(['pub_rec']).groupBy('pub_rec').count().show()

+-------+-------+
|pub_rec|  count|
+-------+-------+
|      7|    302|
|     15|      4|
|     11|     39|
|      3|   6827|
|      8|    111|
|     22|      1|
|     28|      9|
|     16|     14|
|     52|      1|
|      0|1196699|
|     47|      1|
|   null|     27|
|      5|   1300|
|     31|      1|
|     61|      5|
|     17|      2|
|     46|      1|
|      6|    629|
|     19|     16|
|     25|      1|
+-------+-------+
only showing top 20 rows



In [24]:
raw_LendingClubWeb_df.select(['inq_last_6mths']).groupBy('inq_last_6mths').count().show()

+--------------+------+
|inq_last_6mths| count|
+--------------+------+
|             3| 31289|
|             0|925659|
|          null|    27|
|             5|  2400|
|         18.24|     1|
|             1|356887|
|             4|  7274|
|             2|108929|
+--------------+------+



In [25]:
raw_LendingClubWeb_df.select(['settlement_status']).groupBy('settlement_status').count().show()

+-----------------+-------+
|settlement_status|  count|
+-----------------+-------+
|             null|1407055|
|         COMPLETE|   8606|
|           BROKEN|   4116|
|           ACTIVE|  12689|
+-----------------+-------+



##### Get data that contain ONLY Fully Paid and Charged Off

In [26]:
loanPayment_df = raw_LendingClubWeb_df\
.filter((F.col('loan_status') == 'Fully Paid') | ((F.col('loan_status') =='Charged Off')))

In [27]:
loanPayment_df.groupBy('loan_status').count().show()

+-----------+------+
|loan_status| count|
+-----------+------+
| Fully Paid|583755|
|Charged Off|152999|
+-----------+------+



In [28]:
loanPayment_df.count()

736754

##### Get data that contain ONLY related attributes with Business Prespective.

In [29]:
business_df = loanPayment_df.select('annual_inc'\
                                           ,'bc_util'\
                                           ,'inq_fi'\
                                           ,'inq_last_12m'\
                                           ,'home_ownership'\
                                           ,'purpose'\
                                           ,'emp_length'\
                                           ,'revol_bal'\
                                           ,'dti'\
                                           ,'delinq_2yrs'\
                                           ,'pub_rec_bankruptcies'\
                                           ,'pub_rec'\
                                           ,'open_rv_24m'\
                                           ,'mort_acc'\
                                           ,'num_actv_bc_tl'\
                                           ,'num_actv_rev_tl'\
                                           ,'num_il_tl'\
                                           ,'num_tl_90g_dpd_24m'\
                                           ,'int_rate'\
                                           ,'inq_last_6mths'\
                                           ,'term'\
                                           ,'installment'\
                                           ,'total_rev_hi_lim'\
                                           ,'total_bal_il'\
                                           ,'total_bal_ex_mort'\
                                           ,'total_acc'\
                                           ,'tot_cur_bal'\
                                           ,'loan_amnt'\
                                           ,'loan_status'\
                                           ,'verification_status'\
                                           ,'collections_12_mths_ex_med'\
                                           ,'chargeoff_within_12_mths'\
                                           ,'il_util'\
                                           ,'last_pymnt_amnt'\
                                           #,'last_pymnt_d'\
                                           ,'out_prncp_inv'\
                                           ,'out_prncp'\
                                           ,'total_pymnt_inv'\
                                           ,'total_pymnt'\
                                           ,'grade')
                                           #,'sec_app_collections_12_mths_ex_med'\
                                           #,'sec_app_chargeoff_within_12_mths'\
                                           #,'settlement_term'\
                                           #,'settlement_amount'\
                                           #,'settlement_status'

In [30]:
business_df.count()

736754

In [31]:
len(business_df.columns)

39

##### Get Data Profiling

In [32]:
import pandas as pd
business_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
annual_inc,736754,79498.49264390828,78094.7518701319,0,99999.84
bc_util,727864,55.109908993987595,28.903095321098558,0,99.9
inq_fi,736698,1.0650497218670336,5.890278008272024,0,9
inq_last_12m,736697,2.2510679424512383,21.116503762814478,0,9
home_ownership,736754,None,None,ANY,RENT
purpose,736754,None,None,and also pay off some credit card debt. I wi...,wedding
emp_length,736754,None,None,1 year,n/a
revol_bal,736753,16101.816257280256,22819.366258659797,0,99991
dti,736282,18.711086881231356,13.79426027607045,-1,Debt consolidation


### สรุปการดำเนินงานของ Data Understanding: เลือกเฉพาะบาง Attributes และ Row ที่เป็น Charge Off/Fully Charge

### 3) Data Preparation

##### Get Data with No Null

In [33]:
no_null_df = business_df.dropna(how='any')

In [34]:
#no_null_df.count()

##### Get Data with No "wedding"

In [35]:
from pyspark.sql.functions import *

In [36]:
no_wedding_df = no_null_df.filter(F.col('purpose') != 'wedding')

In [37]:
no_wedding_df.select(['purpose']).groupBy('purpose').count().show()

+------------------+------+
|           purpose| count|
+------------------+------+
|             other| 41383|
|    small_business|  6552|
|debt_consolidation|360885|
|       credit_card|130378|
|            moving|  4877|
|          vacation|  5049|
|  renewable_energy|   445|
|             house|  3758|
|               car|  6838|
|    major_purchase| 14407|
|           medical|  8449|
|  home_improvement| 44688|
+------------------+------+



In [38]:
no_wedding_df.count()

627709

### นับจำนวนข้อมูลหลัง Cleansing ค่า 'Null'

In [39]:
fitmem_no_null_df = no_wedding_df.repartition(60)

In [40]:
cached_no_null_df = fitmem_no_null_df.cache()

In [41]:
cached_no_null_df.count()

627709

### Remove '%' และ Extract Month

In [42]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

### Remove '%' 

In [43]:
def f_removepercent(origin):
    return origin.rstrip('%')

In [44]:
removepercent = udf(lambda x: f_removepercent(x),StringType())

### Remove 'Wedding'

In [45]:
#def f_wedding(origin):
    #return origin.rstrip('wedding')

In [46]:
#removewedding = udf(lambda x: f_wedding(x),StringType())

### Extract Month

In [47]:
def f_exrtractmonth(origin):
    return origin.split('-')[0]

In [48]:
exrtractmonth = udf(lambda x: f_exrtractmonth(x),StringType())

### Replace'n/a' ใน Field 'emp_length' ด้วย 'No employ'

In [49]:
def python_treatNA(origin):
    if origin == 'n/a':
        new = 'NotEmployed'
    else:
        new = origin
    return new

In [50]:
treatNA = udf(lambda x: python_treatNA(x),StringType())

### เปลี่ยนแปลง Type ของข้อมูล

In [51]:
from pyspark.sql.functions import col

In [52]:
crunched_df = cached_no_null_df.\
withColumn('emp_length',treatNA(cached_no_null_df['emp_length'])).\
withColumn('int_rate',removepercent(cached_no_null_df['int_rate']).cast(DoubleType())).\
withColumn('dti',cached_no_null_df['dti'].cast(DoubleType())).\
withColumn('revol_bal',cached_no_null_df['revol_bal'].cast(DoubleType())).\
withColumn('pub_rec',cached_no_null_df['pub_rec'].cast(DoubleType())).\
withColumn('total_bal_il',cached_no_null_df['total_bal_il'].cast(DoubleType())).\
withColumn('tot_cur_bal',cached_no_null_df['tot_cur_bal'].cast(DoubleType())).\
withColumn('total_acc',cached_no_null_df['total_acc'].cast(DoubleType())).\
withColumn('total_bal_ex_mort',cached_no_null_df['total_bal_ex_mort'].cast(DoubleType())).\
withColumn('total_rev_hi_lim',cached_no_null_df['total_rev_hi_lim'].cast(DoubleType())).\
withColumn('num_actv_rev_tl',cached_no_null_df['num_actv_rev_tl'].cast(DoubleType())).\
withColumn('num_actv_bc_tl',cached_no_null_df['num_actv_bc_tl'].cast(DoubleType())).\
withColumn('num_il_tl',cached_no_null_df['num_il_tl'].cast(DoubleType())).\
withColumn('pub_rec_bankruptcies',cached_no_null_df['pub_rec_bankruptcies'].cast(DoubleType())).\
withColumn('delinq_2yrs',cached_no_null_df['delinq_2yrs'].cast(DoubleType())).\
withColumn('open_rv_24m',cached_no_null_df['open_rv_24m'].cast(DoubleType())).\
withColumn('num_tl_90g_dpd_24m',cached_no_null_df['num_tl_90g_dpd_24m'].cast(DoubleType())).\
withColumn('inq_last_6mths',cached_no_null_df['inq_last_6mths'].cast(DoubleType())).\
withColumn('bc_util',cached_no_null_df['bc_util'].cast(DoubleType())).\
withColumn('mort_acc',cached_no_null_df['mort_acc'].cast(DoubleType())).\
withColumn('inq_fi',cached_no_null_df['inq_fi'].cast(DoubleType())).\
withColumn('last_pymnt_amnt',cached_no_null_df['last_pymnt_amnt'].cast(DoubleType())).\
withColumn('out_prncp_inv',cached_no_null_df['out_prncp_inv'].cast(DoubleType())).\
withColumn('out_prncp',cached_no_null_df['out_prncp'].cast(DoubleType())).\
withColumn('total_pymnt_inv',cached_no_null_df['total_pymnt_inv'].cast(DoubleType())).\
withColumn('total_pymnt',cached_no_null_df['total_pymnt'].cast(DoubleType())).\
withColumn('il_util',cached_no_null_df['il_util'].cast(DoubleType())).\
withColumn('chargeoff_within_12_mths',cached_no_null_df['chargeoff_within_12_mths'].cast(DoubleType())).\
withColumn('collections_12_mths_ex_med',cached_no_null_df['collections_12_mths_ex_med'].cast(DoubleType())).\
withColumn('loan_amnt',cached_no_null_df['loan_amnt'].cast(DoubleType())).\
withColumn('inq_last_12m',cached_no_null_df['inq_last_12m'].cast(DoubleType())).\
withColumn('installment',cached_no_null_df['installment'].cast(DoubleType())).\
withColumn('annual_inc',cached_no_null_df['annual_inc'].cast(DoubleType()))

In [53]:
crunched_df.printSchema()

root
 |-- annual_inc: double (nullable = true)
 |-- bc_util: double (nullable = true)
 |-- inq_fi: double (nullable = true)
 |-- inq_last_12m: double (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- revol_bal: double (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: double (nullable = true)
 |-- pub_rec_bankruptcies: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- open_rv_24m: double (nullable = true)
 |-- mort_acc: double (nullable = true)
 |-- num_actv_bc_tl: double (nullable = true)
 |-- num_actv_rev_tl: double (nullable = true)
 |-- num_il_tl: double (nullable = true)
 |-- num_tl_90g_dpd_24m: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- inq_last_6mths: double (nullable = true)
 |-- term: string (nullable = true)
 |-- installment: double (nullable = true)
 |-- total_rev_hi_lim: double (nullable = true)
 |-- total_bal_

### เช็ค 'emp_length' หลัง Replace value

In [54]:
crunched_df.select(crunched_df['emp_length']).distinct().show()

+-----------+
| emp_length|
+-----------+
|    5 years|
|    9 years|
|     1 year|
|NotEmployed|
|    2 years|
|    7 years|
|    8 years|
|    4 years|
|    6 years|
|    3 years|
|  10+ years|
|   < 1 year|
+-----------+



In [55]:
crunched_df.select(crunched_df['purpose']).distinct().show()

+------------------+
|           purpose|
+------------------+
|             other|
|    small_business|
|debt_consolidation|
|       credit_card|
|            moving|
|          vacation|
|  renewable_energy|
|             house|
|               car|
|    major_purchase|
|           medical|
|  home_improvement|
+------------------+



### Normalization 'annual_inc'

In [56]:
from pyspark.sql.functions import *
max_annual_inc = crunched_df.select(max('annual_inc')).collect()[0][0]
min_annual_inc = crunched_df.select(min('annual_inc')).collect()[0][0]

In [57]:
def t_annual_inc(origin):
    return ((origin-min_annual_inc)/(max_annual_inc-min_annual_inc))

In [58]:
n_annual_inc = udf(lambda x: t_annual_inc(x),DoubleType())

### Normalization 'loan_amnt'

In [59]:
#max_loan_amt = crunched_df.select(max('loan_amnt')).collect()[0][0]
#min_loan_amt = crunched_df.select(min('loan_amnt')).collect()[0][0]

In [60]:
#def t_loan_amt(origin):
    #return ((origin-min_loan_amt)/(max_loan_amt-min_loan_amt))

In [61]:
#n_loan_amt = udf(lambda x: t_loan_amt(x),DoubleType())

### Normalization 'revol_bal'

In [62]:
max_revol_bal = crunched_df.select(max('revol_bal')).collect()[0][0]
min_revol_bal = crunched_df.select(min('revol_bal')).collect()[0][0]

In [63]:
def t_revol_bal(origin):
    return ((origin-min_revol_bal)/(max_revol_bal-min_revol_bal))

In [64]:
n_revol_bal = udf(lambda x: t_revol_bal(x),DoubleType())

### Normalization 'tot_cur_bal'

In [65]:
max_tot_cur_bal = crunched_df.select(max('tot_cur_bal')).collect()[0][0]
min_tot_cur_bal = crunched_df.select(min('tot_cur_bal')).collect()[0][0]

In [66]:
def t_tot_cur_bal(origin):
    return ((origin-min_tot_cur_bal)/(max_tot_cur_bal-min_tot_cur_bal))

In [67]:
n_tot_cur_bal = udf(lambda x: t_tot_cur_bal(x),DoubleType())

### Normalization 'total_rev_hi_lim'

In [68]:
max_total_rev_hi_lim = crunched_df.select(max('total_rev_hi_lim')).collect()[0][0]
min_total_rev_hi_lim = crunched_df.select(min('total_rev_hi_lim')).collect()[0][0]

In [69]:
def t_total_rev_hi_lim(origin):
    return ((origin-min_total_rev_hi_lim)/(max_total_rev_hi_lim-min_total_rev_hi_lim))

In [70]:
n_total_rev_hi_lim = udf(lambda x: t_total_rev_hi_lim(x),DoubleType())

### Normalization 'total_bal_ex_mort'

In [71]:
max_total_bal_ex_mort = crunched_df.select(max('total_bal_ex_mort')).collect()[0][0]
min_total_bal_ex_mort = crunched_df.select(min('total_bal_ex_mort')).collect()[0][0]

In [72]:
def t_total_bal_ex_mort(origin):
    return ((origin-min_total_bal_ex_mort)/(max_total_bal_ex_mort-min_total_bal_ex_mort))

In [73]:
n_total_bal_ex_mort = udf(lambda x: t_total_bal_ex_mort(x),DoubleType())

### Normalization 'total_bal_il'

In [74]:
max_total_bal_il = crunched_df.select(max('total_bal_il')).collect()[0][0]
min_total_bal_il = crunched_df.select(min('total_bal_il')).collect()[0][0]

In [75]:
def t_total_bal_il(origin):
    return ((origin-min_total_bal_il)/(max_total_bal_il-min_total_bal_il))

In [76]:
n_total_bal_il = udf(lambda x: t_total_bal_il(x),DoubleType())

### รวมตัวแปรที่ทำการ Normalized แล้ว

In [77]:
normalized_df = crunched_df.\
withColumn('annual_inc',n_annual_inc(crunched_df['annual_inc'])).\
withColumn('revol_bal',n_revol_bal(crunched_df['revol_bal'])).\
withColumn('tot_cur_bal',n_tot_cur_bal(crunched_df['tot_cur_bal'])).\
withColumn('total_rev_hi_lim',n_total_rev_hi_lim(crunched_df['total_rev_hi_lim'])).\
withColumn('total_bal_il',n_total_bal_il(crunched_df['total_bal_il'])).\
withColumn('total_bal_ex_mort',n_total_bal_ex_mort(crunched_df['total_bal_ex_mort']))

#withColumn('loan_amnt',n_loan_amt(crunched_df['loan_amnt'])).\

In [78]:
#ดูข้อมูลที่มีการทำ Normalization (ให้ลองเปลี่ยน orderby เป็น GroupBy)

In [79]:
normalized_df.select(['grade'\
                      ,'annual_inc'\
                      ,'revol_bal'\
                      ,'tot_cur_bal'\
                      ,'revol_bal'\
                      ,'total_bal_il'\
                      ,'total_rev_hi_lim']).orderBy(['grade']).show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|grade|          annual_inc|           revol_bal|         tot_cur_bal|           revol_bal|        total_bal_il|    total_rev_hi_lim|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    A|0.006369652970542614| 0.03129385855059931| 0.06936660561996925| 0.03129385855059931|0.005742907505603633| 0.05334769544758773|
|    A|0.005157917816988088|0.012812299244831007| 0.03385630738738501|0.012812299244831007|0.006604670056652908| 0.04422019388854243|
|    A|0.012010489030192993|0.022619917021929167| 0.07332656016185088|0.022619917021929167|0.025018678775832928| 0.02006916491864618|
|    A| 0.00783209194897049|0.007586842676495376| 0.07129093563062855|0.007586842676495376| 0.03209267574370543|0.006633029083281365|
|    A|0.008354391584123303|0.012617201471845722| 0.0140813280

In [80]:
normalized_df.count()

627709

In [81]:
normalized_filtered_df = normalized_df\
#.filter(F.col('loan_status') == 'Fully Paid').\
#union(normalized_df.filter(F.col('loan_status') == 'Charged Off'))

In [82]:
normalized_filtered_df.count()

627709

In [83]:
#normalized_filtered_df.select('loan_status').distinct().show()

### สร้าง Dataframe โดยใช้ Pandas และให้แสดงข้อมูลหลัง Normalized

In [84]:
pd.DataFrame(normalized_df.describe().take(6),\
             columns=normalized_df.describe().columns).transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
annual_inc,627709,0.008462566145329769,0.008274208621998924,0.0,1.0
bc_util,627709,55.70084688924324,28.831685669621603,0.0,252.3
inq_fi,627709,1.1189500230202212,1.5859378645745754,0.0,48.0
inq_last_12m,627709,2.3041584555900902,2.528224936897044,0.0,67.0
home_ownership,627709,None,None,ANY,RENT
purpose,627709,None,None,car,vacation
emp_length,627709,None,None,1 year,NotEmployed
revol_bal,627709,0.014265257680731206,0.019837633181117042,0.0,1.0
dti,627709,19.848822623221896,13.332388424437132,-1.0,999.0


### กำจัดค่า 'Null' จากการทำ Data Prep

In [85]:
data_no_missing_df = normalized_filtered_df.dropna(how='any')

In [86]:
data_no_missing_df.count()

627709

In [87]:
data_no_missing_df.printSchema()

root
 |-- annual_inc: double (nullable = true)
 |-- bc_util: double (nullable = true)
 |-- inq_fi: double (nullable = true)
 |-- inq_last_12m: double (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- revol_bal: double (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: double (nullable = true)
 |-- pub_rec_bankruptcies: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- open_rv_24m: double (nullable = true)
 |-- mort_acc: double (nullable = true)
 |-- num_actv_bc_tl: double (nullable = true)
 |-- num_actv_rev_tl: double (nullable = true)
 |-- num_il_tl: double (nullable = true)
 |-- num_tl_90g_dpd_24m: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- inq_last_6mths: double (nullable = true)
 |-- term: string (nullable = true)
 |-- installment: double (nullable = true)
 |-- total_rev_hi_lim: double (nullable = true)
 |-- total_bal_

### 4) ML Modeling

In [88]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.feature import StringIndexer, VectorAssembler, \
OneHotEncoder, VectorIndexer
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, \
BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression, GBTClassifier, \
NaiveBayes, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.ml.clustering import *

In [89]:
#เปลี่ยนข้อมูลที่ต้องการ (Field) จาก Categorical เป็น Numerical

In [90]:
labelIndexer = StringIndexer(inputCol='loan_status',outputCol='indexedLabel')

In [91]:
gradeIndexer = StringIndexer(inputCol='grade',outputCol='gradeIndexed')
gradeOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='gradeIndexed',\
                                  outputCol='gradeVec')

In [92]:
homeIndexer = StringIndexer(inputCol='home_ownership',outputCol='homeIndexed')
homeOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='homeIndexed',\
                                  outputCol='homeVec')

In [93]:
purposeIndexer = StringIndexer(inputCol='purpose',outputCol='purposeIndexed')
purposeOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='purposeIndexed',\
                                  outputCol='purposeVec')

In [94]:
emp_lengthIndexer = StringIndexer(inputCol='emp_length',outputCol='emp_lengthIndexed')
emp_lengthOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='emp_lengthIndexed',\
                                  outputCol='emp_lengthVec')

In [95]:
verification_statusIndexer = StringIndexer(inputCol='verification_status',outputCol='verification_statusIndexed')
verification_statusOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='verification_statusIndexed',\
                                  outputCol='verification_statusVec')

### เลือก Feature ที่จะใช้ในการ Train

In [96]:
featureAssembler = VectorAssembler(inputCols=['annual_inc'\
                                              #,'bc_util'\
                                              #,'inq_fi'\
                                              #,'inq_last_12m'\
                                              #,'home_ownership'\
                                              #,'purpose'\
                                              #,'emp_length'\
                                              ,'installment'\
                                              #,'total_rev_hi_lim'\
                                              ,'loan_amnt'\
                                              #,'loan_status'\
                                              #,'verification_status'\
                                              ,'total_pymnt'\
                                              ,'gradeVec'\
                                              ,'homeVec'\
                                              ,'emp_lengthVec'\
                                              ,'purposeVec'\
                                              ,'verification_statusVec']\
                                   ,outputCol='***features')

In [97]:
selected_attr_list = ['annual_inc'\
                                              #,'bc_util'\
                                              #,'inq_fi'\
                                              #,'inq_last_12m'\
                                              #,'home_ownership'\
                                              #,'purpose'\
                                              #,'emp_length'\
                                              ,'installment'\
                                              #,'total_rev_hi_lim'\
                                              ,'loan_amnt'\
                                              #,'loan_status'\
                                              #,'verification_status'\
                                              ,'total_pymnt'\
                                              ,'grade'\
                                              ,'home'\
                                              ,'emp_length'\
                                              ,'purpose'\
                                              ,'verification_status']

# Modeling with DecisionTree

In [98]:
dt = DecisionTreeClassifier(featuresCol='***features',labelCol='indexedLabel')

In [99]:
#ทำ Sequencing ลำดับของการทำ Modeling

In [100]:
pipeline_dt = Pipeline().setStages([gradeIndexer,gradeOneHotEncoder,\
                                    homeIndexer,homeOneHotEncoder,\
                                    emp_lengthIndexer,emp_lengthOneHotEncoder,\
                                    purposeIndexer,purposeOneHotEncoder,\
                                    verification_statusIndexer,verification_statusOneHotEncoder,\
                                    labelIndexer,\
                                    featureAssembler,\
                                    dt])

### แบ่งข้อมูลเป็น Training และ Testing

In [101]:
#แบ่งข้อมูลเป็น Training และ Testing (ุ60% - 40%) โดย Random จากข้อมูลที่เหลืออยู่

In [102]:
training_dt, test_dt = normalized_df.randomSplit([0.6,0.4], seed = 13)

In [103]:
#ดูการการกระจุกตัวของ Label จากการ Random เพื่อดู Imbalance Data

In [104]:
training_dt.groupBy('loan_status').count().orderBy(['loan_status']).show()

+-----------+------+
|loan_status| count|
+-----------+------+
|Charged Off| 78630|
| Fully Paid|297691|
+-----------+------+



In [105]:
training_dt.groupBy('grade').count().show()

+-----+------+
|grade| count|
+-----+------+
|    F|  7665|
|    E| 22275|
|    B|111733|
|    D| 52771|
|    C|113368|
|    A| 66275|
|    G|  2234|
+-----+------+



In [106]:
training_dt.count()

376321

In [107]:
#แบ่งข้อมูลเป็น Training และ Testing (ุ60%) โดย Random จาก Label ในแต่ละ Catergory 

In [108]:
#training, test = normalized_df\
#.filter(F.col('grade') == 'A').sample(False,0.4,42).\
#union(normalized_df.filter(F.col('grade') =='B').\
      #sample(False,0.4,42)).\
#union(normalized_df.filter(F.col('grade') =='C').\
      #ample(False,0.4,42)).\
#union(normalized_df.filter(F.col('grade') =='D').\
      #sample(False,0.4,42)).\
#union(normalized_df.filter(F.col('grade') =='E')).\
#union(normalized_df.filter(F.col('grade') =='F')).\
#union(normalized_df.filter(F.col('grade') =='G')).\
#randomSplit([0.4,0.2], seed = 12)

In [109]:
start_time_dt = t.time()

In [110]:
model_dt = pipeline_dt.fit(training_dt)

In [111]:
duration_dt = t.time() - start_time_dt

In [112]:
print('ใช้เวลา Distributed Training {0} วินาที'.format(duration_dt))

ใช้เวลา Distributed Training 51.33565664291382 วินาที


In [113]:
spark_fullLabel_func = udf(lambda x: "Fully Paid" if x == 0.0 else "Charged Off")

In [114]:
result_dt = model_dt.transform(test_dt.dropna()).withColumn('full_prediction',spark_fullLabel_func(F.col('prediction')))

In [115]:
#ดูการการกระจุกตัวของ Label จากการ Random เพื่อดู Imbalance Data (ข้อมูล Test)

In [116]:
result_dt.groupBy(['loan_status','indexedLabel']).count().show()

+-----------+------------+------+
|loan_status|indexedLabel| count|
+-----------+------------+------+
| Fully Paid|         0.0|198762|
|Charged Off|         1.0| 52626|
+-----------+------------+------+



In [117]:
result_dt.count()

251388

In [118]:
result_dt

DataFrame[annual_inc: double, bc_util: double, inq_fi: double, inq_last_12m: double, home_ownership: string, purpose: string, emp_length: string, revol_bal: double, dti: double, delinq_2yrs: double, pub_rec_bankruptcies: double, pub_rec: double, open_rv_24m: double, mort_acc: double, num_actv_bc_tl: double, num_actv_rev_tl: double, num_il_tl: double, num_tl_90g_dpd_24m: double, int_rate: double, inq_last_6mths: double, term: string, installment: double, total_rev_hi_lim: double, total_bal_il: double, total_bal_ex_mort: double, total_acc: double, tot_cur_bal: double, loan_amnt: double, loan_status: string, verification_status: string, collections_12_mths_ex_med: double, chargeoff_within_12_mths: double, il_util: double, last_pymnt_amnt: double, out_prncp_inv: double, out_prncp: double, total_pymnt_inv: double, total_pymnt: double, grade: string, gradeIndexed: double, gradeVec: vector, homeIndexed: double, homeVec: vector, emp_lengthIndexed: double, emp_lengthVec: vector, purposeIndexed:

In [119]:
num_result_dt = result_dt.count()

# การประเมิน Charge-Off [Aekanun]

In [120]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|  

In [121]:
num_Actual_ChargeOff = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Charged Off').count()

In [122]:
num_Actual_ChargeOff

52626

In [123]:
#ตรวจสอบความถูกต้องจากการ Predict ตรงกับข้อมูลที่เป็น IndexLabel หรือไม่ (เนื่องจากมีการทำ indexer ค่าที่ได้จึงเป็น Numerical)

### Positive = Charged Off

In [124]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|  

In [125]:
num_False_FullyPaid = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Charged Off').count()

In [126]:
(num_False_FullyPaid/num_Actual_ChargeOff)*100

17.13791661916163

In [127]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|  

In [128]:
num_True_ChargeOff = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Charged Off').count()

In [129]:
(num_True_ChargeOff/num_Actual_ChargeOff)*100

82.86208338083837

# ------------- สิ้นสุดการประเมิน ---------------------------

# การประเมิน Fully Paid [Aekanun]

In [130]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|  

In [131]:
num_Actual_FullyPaid = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Fully Paid').count()

In [132]:
num_Actual_FullyPaid

198762

In [133]:
#ตรวจสอบความถูกต้องจากการ Predict ตรงกับข้อมูลที่เป็น IndexLabel หรือไม่ (เนื่องจากมีการทำ indexer ค่าที่ได้จึงเป็น Numerical)

### Negative = Fully Paid

In [134]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|  

In [135]:
num_False_ChargedOff = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').count()

In [136]:
(num_False_ChargedOff/num_Actual_FullyPaid)*100

0.09659794125637698

In [137]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|  

In [138]:
num_True_FullyPaid = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').count()

In [139]:
(num_True_FullyPaid/num_Actual_FullyPaid)*100

99.90340205874362

# ------------- สิ้นสุดการประเมิน ---------------------------

# การประเมิน Accuracy ***

In [140]:
num_Actual_FullyPaid + num_Actual_ChargeOff

251388

In [141]:
(num_True_FullyPaid+num_True_ChargeOff)/(num_Actual_FullyPaid + num_Actual_ChargeOff)

0.9633594284532276

In [142]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [143]:
evaluator_DT = MulticlassClassificationEvaluator(predictionCol="prediction",\
                                              labelCol='indexedLabel', metricName='accuracy')
evaluator_DT.evaluate(result_dt)*100

96.33594284532276

# ------------- สิ้นสุดการประเมิน --------------------------- ***

In [144]:
#model_dt.write().overwrite().save('gs://grizzy-lab/loanpayment_dtModel')

# Modeling with RandomForest

In [145]:
RF = RandomForestClassifier(featuresCol='***features',labelCol='indexedLabel')

In [146]:
pipeline_RF = Pipeline().setStages([gradeIndexer,gradeOneHotEncoder,\
                                    homeIndexer,homeOneHotEncoder,\
                                    emp_lengthIndexer,emp_lengthOneHotEncoder,\
                                    purposeIndexer,purposeOneHotEncoder,\
                                    verification_statusIndexer,verification_statusOneHotEncoder,\
                                    labelIndexer,\
                                    featureAssembler,\
                                    RF])

In [147]:
#training_RF, test_RF = normalized_df.randomSplit([0.6,0.4])

In [148]:
training_RF = training_dt

In [149]:
test_RF = test_dt

In [150]:
training_RF.groupBy('loan_status').count().orderBy(['loan_status']).show()

+-----------+------+
|loan_status| count|
+-----------+------+
|Charged Off| 78630|
| Fully Paid|297691|
+-----------+------+



In [151]:
training_RF.count()

376321

In [152]:
start_time_rf = t.time()

In [153]:
model_RF = pipeline_RF.fit(training_RF)

In [154]:
duration_rf = t.time() - start_time_rf

In [155]:
print('ใช้เวลา Distributed Training {0} วินาที'.format(duration_rf))

ใช้เวลา Distributed Training 43.40709948539734 วินาที


In [156]:
result_RF = model_RF.transform(test_RF.dropna()).withColumn('full_prediction',spark_fullLabel_func(F.col('prediction')))

In [157]:
result_RF.groupBy('loan_status').count().show()

+-----------+------+
|loan_status| count|
+-----------+------+
| Fully Paid|198762|
|Charged Off| 52626|
+-----------+------+



In [158]:
result_RF.count()

251388

In [159]:
result_RF

DataFrame[annual_inc: double, bc_util: double, inq_fi: double, inq_last_12m: double, home_ownership: string, purpose: string, emp_length: string, revol_bal: double, dti: double, delinq_2yrs: double, pub_rec_bankruptcies: double, pub_rec: double, open_rv_24m: double, mort_acc: double, num_actv_bc_tl: double, num_actv_rev_tl: double, num_il_tl: double, num_tl_90g_dpd_24m: double, int_rate: double, inq_last_6mths: double, term: string, installment: double, total_rev_hi_lim: double, total_bal_il: double, total_bal_ex_mort: double, total_acc: double, tot_cur_bal: double, loan_amnt: double, loan_status: string, verification_status: string, collections_12_mths_ex_med: double, chargeoff_within_12_mths: double, il_util: double, last_pymnt_amnt: double, out_prncp_inv: double, out_prncp: double, total_pymnt_inv: double, total_pymnt: double, grade: string, gradeIndexed: double, gradeVec: vector, homeIndexed: double, homeVec: vector, emp_lengthIndexed: double, emp_lengthVec: vector, purposeIndexed:

# การประเมิน Charge-Off [Aekanun]

In [160]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|  

In [161]:
num_Actual_ChargeOff = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Charged Off').count()

In [162]:
num_Actual_ChargeOff

52626

In [163]:
#ตรวจสอบความถูกต้องจากการ Predict ตรงกับข้อมูลที่เป็น IndexLabel หรือไม่ (เนื่องจากมีการทำ indexer ค่าที่ได้จึงเป็น Numerical)

### Positive = Charged Off

In [164]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|  

In [165]:
num_False_FullyPaid = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Charged Off').count()

In [166]:
num_False_FullyPaid

44495

In [167]:
(num_False_FullyPaid/num_Actual_ChargeOff)*100

84.54946224299775

In [168]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|  

In [169]:
num_True_ChargeOff = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Charged Off').count()

In [170]:
(num_True_ChargeOff/num_Actual_ChargeOff)*100

15.450537757002241

# ------------- สิ้นสุดการประเมิน ---------------------------

# การประเมิน Fully Paid [Aekanun]

In [171]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|  

In [172]:
num_Actual_FullyPaid = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Fully Paid').count()

In [173]:
num_Actual_FullyPaid

198762

In [174]:
#ตรวจสอบความถูกต้องจากการ Predict ตรงกับข้อมูลที่เป็น IndexLabel หรือไม่ (เนื่องจากมีการทำ indexer ค่าที่ได้จึงเป็น Numerical)

### Negative = Fully Paid

In [175]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
+-----------+------------+----------+---------------+



In [176]:
num_False_ChargedOff = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').count()

In [177]:
(num_False_ChargedOff/num_Actual_FullyPaid)*100

0.0

In [178]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|  

In [179]:
num_True_FullyPaid = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').count()

In [180]:
(num_True_FullyPaid/num_Actual_FullyPaid)*100

100.0

# ------------- สิ้นสุดการประเมิน ---------------------------

In [181]:
num_Actual_FullyPaid + num_Actual_ChargeOff

251388

In [182]:
(num_True_FullyPaid+num_True_ChargeOff)/(num_Actual_FullyPaid + num_Actual_ChargeOff)

0.8230026890702818

In [183]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [184]:
evaluator_RF = MulticlassClassificationEvaluator(predictionCol="prediction",\
                                              labelCol='indexedLabel', metricName='accuracy')
evaluator_RF.evaluate(result_RF)*100

82.30026890702818

# Modeling with Multi layer Perceptron (ANN)

In [185]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [186]:
pipeline_mlp = Pipeline(stages=[gradeIndexer,gradeOneHotEncoder,\
                                    homeIndexer,homeOneHotEncoder,\
                                    emp_lengthIndexer,emp_lengthOneHotEncoder,\
                                    purposeIndexer,purposeOneHotEncoder,\
                                    verification_statusIndexer,verification_statusOneHotEncoder,\
                                    labelIndexer,\
                                    featureAssembler])

In [187]:
#training_mlp, test_mlp = normalized_df.randomSplit([0.6,0.4])

In [188]:
training_mlp = training_dt

In [189]:
test_mlp = test_dt

In [190]:
train_df_features = pipeline_mlp.fit(training_mlp).transform(training_mlp)

In [191]:
layers = [train_df_features.schema["***features"].metadata["ml_attr"]["num_attrs"],20 ,10, 2]

In [192]:
layers

[43, 20, 10, 2]

### Implementation of MLP

In [193]:
clf = MultilayerPerceptronClassifier(labelCol='indexedLabel'\
                                    ,featuresCol='***features'\
                                    ,layers = layers)

In [194]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [195]:
pipeline_mlp_test = Pipeline(stages=[gradeIndexer,gradeOneHotEncoder,\
                                    homeIndexer,homeOneHotEncoder,\
                                    emp_lengthIndexer,emp_lengthOneHotEncoder,\
                                    purposeIndexer,purposeOneHotEncoder,\
                                    verification_statusIndexer,verification_statusOneHotEncoder,\
                                    labelIndexer,\
                                    featureAssembler,
                                    clf])

In [196]:
#training_mlp_test, test_mlp_test = normalized_df.randomSplit([0.6,0.4])

In [197]:
training_mlp.groupBy('loan_status').count().orderBy(['loan_status']).show()

+-----------+------+
|loan_status| count|
+-----------+------+
|Charged Off| 78630|
| Fully Paid|297691|
+-----------+------+



In [198]:
training_mlp.count()

376321

In [199]:
start_time_mlp = t.time()

In [200]:
model_mlp = pipeline_mlp_test.fit(training_mlp)

In [201]:
duration_mlp = t.time() - start_time_mlp

In [202]:
print('ใช้เวลา Distributed Training {0} วินาที'.format(duration_mlp))

ใช้เวลา Distributed Training 35.44558811187744 วินาที


In [203]:
result_mlp = model_mlp.transform(test_mlp.dropna())\
.withColumn('full_prediction',spark_fullLabel_func(F.col('prediction')))

In [204]:
result_mlp.groupBy('loan_status').count().show()

+-----------+------+
|loan_status| count|
+-----------+------+
| Fully Paid|198762|
|Charged Off| 52626|
+-----------+------+



In [205]:
result_mlp.count()

251388

In [206]:
result_mlp

DataFrame[annual_inc: double, bc_util: double, inq_fi: double, inq_last_12m: double, home_ownership: string, purpose: string, emp_length: string, revol_bal: double, dti: double, delinq_2yrs: double, pub_rec_bankruptcies: double, pub_rec: double, open_rv_24m: double, mort_acc: double, num_actv_bc_tl: double, num_actv_rev_tl: double, num_il_tl: double, num_tl_90g_dpd_24m: double, int_rate: double, inq_last_6mths: double, term: string, installment: double, total_rev_hi_lim: double, total_bal_il: double, total_bal_ex_mort: double, total_acc: double, tot_cur_bal: double, loan_amnt: double, loan_status: string, verification_status: string, collections_12_mths_ex_med: double, chargeoff_within_12_mths: double, il_util: double, last_pymnt_amnt: double, out_prncp_inv: double, out_prncp: double, total_pymnt_inv: double, total_pymnt: double, grade: string, gradeIndexed: double, gradeVec: vector, homeIndexed: double, homeVec: vector, emp_lengthIndexed: double, emp_lengthVec: vector, purposeIndexed:

# การประเมิน Charge-Off [Aekanun]

In [207]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|  

In [208]:
num_Actual_ChargeOff = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Charged Off').count()

In [209]:
num_Actual_ChargeOff

52626

In [210]:
#ตรวจสอบความถูกต้องจากการ Predict ตรงกับข้อมูลที่เป็น IndexLabel หรือไม่ (เนื่องจากมีการทำ indexer ค่าที่ได้จึงเป็น Numerical)

### Positive = Charged Off

In [211]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|  

In [212]:
num_False_FullyPaid = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Charged Off').count()

In [213]:
num_False_FullyPaid

52626

In [214]:
(num_False_FullyPaid/num_Actual_ChargeOff)*100

100.0

In [215]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
+-----------+------------+----------+---------------+



In [216]:
num_True_ChargeOff = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Charged Off').count()

In [217]:
(num_True_ChargeOff/num_Actual_ChargeOff)*100

0.0

# ------------- สิ้นสุดการประเมิน ---------------------------

# การประเมิน Fully Paid [Aekanun]

In [218]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|  

In [219]:
num_Actual_FullyPaid = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Fully Paid').count()

In [220]:
num_Actual_FullyPaid

198762

In [221]:
#ตรวจสอบความถูกต้องจากการ Predict ตรงกับข้อมูลที่เป็น IndexLabel หรือไม่ (เนื่องจากมีการทำ indexer ค่าที่ได้จึงเป็น Numerical)

### Negative = Fully Paid

In [222]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
+-----------+------------+----------+---------------+



In [223]:
num_False_ChargedOff = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').count()

In [224]:
(num_False_ChargedOff/num_Actual_FullyPaid)*100

0.0

In [225]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|  

In [226]:
num_True_FullyPaid = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').count()

In [227]:
(num_True_FullyPaid/num_Actual_FullyPaid)*100

100.0

# ------------- สิ้นสุดการประเมิน ---------------------------

In [228]:
num_Actual_FullyPaid + num_Actual_ChargeOff

251388

In [229]:
(num_True_FullyPaid+num_True_ChargeOff)/(num_Actual_FullyPaid + num_Actual_ChargeOff)

0.7906582653109934

In [230]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [231]:
evaluator_RF = MulticlassClassificationEvaluator(predictionCol="prediction",\
                                              labelCol='indexedLabel', metricName='accuracy')
evaluator_RF.evaluate(result_mlp)*100

79.06582653109933

In [232]:
#model_mlp.save('./modelMLP_bestloanpayment')

In [233]:
! df -h

Filesystem      Size  Used Avail Use% Mounted on
udev            7.9G     0  7.9G   0% /dev
tmpfs           1.6G   29M  1.6G   2% /run
/dev/sda1        79G   14G   63G  18% /
tmpfs           7.9G     0  7.9G   0% /dev/shm
tmpfs           5.0M     0  5.0M   0% /run/lock
tmpfs           7.9G     0  7.9G   0% /sys/fs/cgroup
/dev/sda15      124M  5.7M  119M   5% /boot/efi
tmpfs           1.6G     0  1.6G   0% /run/user/108
tmpfs           1.6G     0  1.6G   0% /run/user/110
tmpfs           1.6G     0  1.6G   0% /run/user/109
tmpfs           1.6G     0  1.6G   0% /run/user/111
tmpfs           1.6G     0  1.6G   0% /run/user/112


In [234]:
! ls -l modelMLP_bestloanpayment

ls: cannot access 'modelMLP_bestloanpayment': No such file or directory
